---
layout: code-post
title: Tafl
tags: [tafl]
description: Implementation of the brandub variant of tafl
---

I'm going to implement the brandub / brandubh / brannumh variant of the viking game 
of tafl / hnefatafl. Eventually I want to do some 
reinforcement learning with this. The fundamental point of the game is that
there is a monarch in a castle that wishes to escape to the forest. The monarch
has a set of defenders to escort them and a set of attackers trying to prevent them
from reaching the forest. In this. particular case, the board is 7x7 with four
defenders and eight attackers. The rules that I'm going by can be found
[here](http://tafl.cyningstan.com/page/171/brandub).

In [1]:
import numpy as np

In [2]:
class UnsettableAttributeError(Exception):
    def __init__(self):
        super().__init__("can't set attribute explicitly once set")
        
        
class InvalidTeamNameError(Exception):
    def __init__(self):
        super().__init__("team name must be 'attack' or 'defense'")
        
        
def initial_only_setter(func):
    def wrapper(self, val):
        if hasattr(self, func.__name__):
            raise UnsettableAttributeError
        else:
            func(self, val)
    return wrapper

def get_brandub_pos(raw_pos):
    """ translate between (i, j) tuple
    and chess style [A-G][1-7] positions """
    num = str(raw_pos[0, 1] + 1)
    i_to_char = {
        0: 'G', 1: 'F', 2: 'E', 3: 'D',
        4: 'C', 5: 'B', 6: 'A'
    }
    return i_to_char[raw_pos[0, 0]] + num

def array_to_tuple(position):
    """ take a (1, 2) numpy array and convert it to a tuple"""
    assert position.shape == (1, 2), "wrong shape"
    return (position[0, 0], position[0, 1])

My initial attempt at this (not present here, maybe in an early commit?) was object
oriented, but it was proving to be a nightmare and hard to divide up responsibilites. 
Instead I'm going to implement a more purefly functional approach. I think this will 
also be helpful in using reinforcement learning because it will be much easier to advance 
the board state and explore trees of possible lines of play. I will create a slew of custom
classes now, but they will be non-mutable and should be easier to instantiate.

There are only three types of pieces, attacking pawns, defending pawns, and the defense's
monarch. The board is 7x7 -- although it is effectively smaller for some pieces -- so that
means the board state should be represented as a 3x7x7 array.

In [3]:
class RawBoard:
    """ A numpy array with a little extra functionality """
    
    def __init__(self, board_array=None, indices=None, skip_checks=False):
        """ board_array must be (3,7,7) with all zero-one entries.
        indices must be list of 3-tuples """
        
        self.__skip_checks = skip_checks
        
        assert board_array is None or indices is None, \
            "one of board_array or indices must be None"
        assert board_array is not None or indices is not None, \
            "one of board_array or indices must not be None"
        
        if board_array is not None:
            if not self.skip_checks:
                assert type(board_array) == np.ndarray, \
                    "board_array must be numpy.ndarray"
                assert board_array.shape == (3, 7, 7), "must have shape (3, 7, 7)"
                assert len(np.unique(board_array)) == 2, "more than 0 and 1 in array"
                assert 0 in np.unique(board_array), "no zeros in board_array"
                assert 1 in np.unique(board_array), "no ones in board_array"
                
            self.__shape = board_array.shape
            self.__raw_board = board_array

            raw_indices = np.where(self.raw_board==1)
            self.__indices = [
                (raw_indices[0][i], raw_indices[1][i], raw_indices[2][i])
                for i in range(len(raw_indices[0]))
            ]
        else:
            if not self.skip_checks:
                assert type(indices) == list, "indices must be list"
                assert len(indices) > 0, "indices cannot be empty"
                assert all(type(ind) == tuple for ind in indices), \
                    "all indices must be tuples"
                assert all(len(ind) == 3 for ind in indices), \
                    "all indices must have length 3"
                assert all(
                    type(i) == int or type(i) == np.int64 
                    for ind in indices 
                    for i in ind), \
                    "all values in each index must be ints"
                assert min(i for ind in indices for i in ind) >= 0, \
                    "all indices must be >= 0"
                assert max(ind[0] for ind in indices) <= 2, \
                    "max value <= 2 for first position"
                assert max(ind[1] for ind in indices) <= 6, \
                    "max value <= 6 for second position"
                assert max(ind[2] for ind in indices) <= 6, \
                    "max value <= 6 for third position"
                
            self.__indices = indices
            board_array = np.zeros((3, 7, 7))
            for ind in indices:
                board_array[ind] = 1
            self.__raw_board = board_array
            
            
    def __eq__(self, other):
        """ determine equality based on nupy matrices"""
        return (self.raw_board == other.raw_board).all()

    
    def __getitem__(self, *args):
        return self.raw_board.__getitem__(*args)
            
        
    @property
    def indices(self):
        return self.__indices
    
        
    @property
    def raw_board(self):
        return self.__raw_board
    
    
    @property
    def shape(self):
        return self.__shape

    
    @property
    def skip_checks(self):
        return self.__skip_checks
    
    
    def flatten(self):
        return self.raw_board.flatten()

In [4]:
class Board:
    """ a 3x7x7 array """
    
    def __init__(self, raw_board, fully_validate_board=True):
        assert type(fully_validate_board) == bool, \
            "fully_validate_board must be bool"
        self.__fully_validate_board = fully_validate_board
        
        assert type(raw_board) == RawBoard, "board must be RawBoard"
        if self.fully_validate_board:
            self.validate_board(raw_board)
        self.__raw_board = raw_board
        
        
    def __eq__(self, other):
        return self.raw_board == other.raw_board
        
        
    def __str__(self):
        return str(self.shadow_pretty)
        
        
    @property
    def positions(self):
        return self.raw_board.indices
        
        
    @property
    def raw_board(self):
        return self.__raw_board
        
        
    @property
    def fully_validate_board(self):
        return self.__fully_validate_board
        
        
    @property
    def shadow(self):
        return self.raw_board[0] + self.raw_board[1] + self.raw_board[2]
    
    
    @property
    def shadow_pretty(self):
        return -1 * self.raw_board[0] + self.raw_board[1] \
            + 2 * self.raw_board[2]
    
    
    @staticmethod
    def validate_board(board):
        """ takes (3, 7, 7) numpy array and validates it as a
        brandub board.
        
        Note: can be in a position that will result in pieces
        being removed.
        """
        assert np.max(board[0] + board[1] + board[2]) == 1, \
            "some tile has more than 1 piece on it"
        assert np.sum(board[0]) <= 8, "more than 8 attacking pawns"
        assert np.sum(board[1]) <= 4, "more than 4 defending pawns"
        assert np.sum(board[2]) <= 1, "more than 1 monarch"
        assert board[0, 3, 3] + board[1, 3, 3] == 0, \
            "no pawns allowed in castle"
        num_pawns_in_forests = \
            board[0, 0, 0] + board[0, 6, 0] \
            + board[0, 0, 6] + board[0, 6, 6] \
            + board[1, 0, 0] + board[1, 6, 0] \
            + board[1, 0, 6] + board[1, 6, 6]
        assert num_pawns_in_forests == 0, "no pawns allowed in forests"
        
        
    def check_victory(self, team):
        """ check for victory for the given team.
        
        team must 'attack' or 'defense'."""
        if team == "defense":
            return self.raw_board[2, 0, 0] + self.raw_board[2, 6, 0] \
                + self.raw_board[2, 0, 6] + self.raw_board[2, 6, 6] == 1
        elif team == "attack":
            return np.sum(self.raw_board[2]) == 0
        else:
            raise InvalidTeamNameError

In [5]:
def get_initial_board(fully_validate_board=True):
    
    indices = [
        (0, 3, 0),
        (0, 3, 1),
        (0, 3, 5),
        (0, 3, 6),
        (0, 0, 3),
        (0, 1, 3),
        (0, 5, 3),
        (0, 6, 3),
        (1, 3, 2),
        (1, 3, 4),
        (1, 2, 3),
        (1, 4, 3),
        (2, 3, 3)
    ]
    
    x = np.zeros((3, 7, 7))
    for i in indices:
        x[i] = 1
    
    raw_board = RawBoard(board_array=x,
                         skip_checks=not fully_validate_board)
    
    return Board(raw_board, fully_validate_board)

In [6]:
board = get_initial_board()
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [-1. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


Now let's determine valid moves for a piece. As a reminder, all pieces move
horizontally or vertically any number of spaces. Pieces cannot jump over
other pieces, but they can jump past the castle. No piece may land on the
empty (or occupied) castle, and only the monarch may land in a forest.

In [7]:
def is_forest(position_2d):
    """ returns whether the first two coords of
    position_2d is a forest"""
    return position_2d[0] in [0, 6] and position_2d[1] in [0, 6]

def is_castle(position_2d):
    """ returns whether the first two coords of position_2d
    is a castle"""
    return position_2d[0] == 3 and position_2d[1] == 3

def find_valid_moves(board, position):
    """ find valid moves for the piece at the given position.
    this assumes there is a piece at the position.
    
    position must be tuple (i, j, k)"""
    is_pawn = position[0] < 2
    
    shadow = board.shadow
    pos_2d = np.array(position[1:])
    
    def check_direction(direction_vector):
        """ get valid moves in the given direction
        
        direction must be +/- [1, 0] or +/- [0, 1] numpy arrays
        """
        
        valid_moves = []
        
        coord = 0 if direction_vector[0] != 0 else 1
        positive_direction = direction_vector[coord] == 1
        end_value = 6 if positive_direction else 0
        
        keep_going = pos_2d[coord] != end_value
        i = 0
        while keep_going:
            i += 1
            
            new_pos = pos_2d + i * direction_vector
            
            # stop if run into a piece
            if shadow[tuple(new_pos)] == 1:
                break
                
            # ignore the castle
            if is_castle(new_pos):
                continue
                
            keep_going = new_pos[coord] != end_value
            
            # if pawn and at the wall, see if its a forest but don't add
            if not keep_going and is_pawn and is_forest(new_pos):
                break
                
            valid_moves.append(tuple(new_pos))
            
        return valid_moves

    direction_vectors = (
        np.array([1, 0]),
        np.array([-1, 0]),
        np.array([0, 1]),
        np.array([0, -1])
    )
    
    return [
        (position[0], vec[0], vec[1])
        for dvec in direction_vectors
        for vec in check_direction(dvec)
    ]

The other main piece of gameplay is determine whether a piece is captured.
Pawns are captured if they are trapped either horizontally or vertically
by any combination of enemy pawns, forests, or empty castles. The attacking
pawns also have to watch out for the occupied castle since only the
defense's monarch can occupy it, while the defense's pawns only have to worry
about the empty castle (is it haunted?). If the monarch is ensconced in the castle,
then they only have to worry if surrounded on all four sides by enemy pawns.
If they are next to the vacant castle (horzonally or vertically) then they only 
have to worry only if surrounded on all three sides by enemy pawns. Otherwise, the
monarch can be captured as if they were a pawn.

In [8]:
def is_piece_captured(raw_board, piece_position):
    """ determine if a piece at (i, j, k) is captured
    given the positions on a (3, 7, 7) numpy array
    raw_board. 
    
    WARNING: This does not validate that the raw_board is a
    vaid raw_board. """
    i = piece_position[1]
    j = piece_position[2]

    topography_inds_corners = [(0, 0), (0, 6), (6, 0), (6, 6)]
    
    if piece_position[0] == 0 or piece_position[0] == 1:
        
        if piece_position[0] == 0:
            enemy_board = raw_board[1] + raw_board[2]
            topography_inds = topography_inds_corners + [(3, 3)]
            
        else:  
            enemy_board = raw_board[0]
            
            if raw_board[2, 3, 3] == 0:
                # no monarch? then castle is threat
                topography_inds = topography_inds_corners + [(3, 3)]
            else:
                topography_inds = topography_inds_corners
        
        topography = np.zeros((7, 7))
        for ind in topography_inds:
            topography[ind] = 1
        bad_things = enemy_board + topography
        
        if i == 0 or i == 6:
            # on top or bottom (can't be in corner)
            return bad_things[i, j-1] == 1 and bad_things[i, j+1] == 1
        elif j == 0 or j == 6:
            # on left or right side (can't be in corner)
            return bad_things[i-1, j] == 1 and bad_things[i+1, j] == 1
        else:
            # otherwise just check
            if bad_things[i, j-1] == 1 and bad_things[i, j+1] == 1:
                return True
            elif bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1:
                return True
            else:
                return False

    elif piece_position[0] == 2:
        
        enemy_board = raw_board[0]
        
        if i != 3 or j != 3:
            topography_inds = topography_inds_corners + [(3, 3)]
        else:
            topography_inds = topography_inds_corners
        
        topography = np.zeros((7, 7))
        for ind in topography_inds:
            topography[ind] = 1
        bad_things = enemy_board + topography
        
        if (i, j) in [(3, 2), (3, 3), (3, 4), (2, 3), (4, 3)]:
            # in or next to castle have to be surrounded
            return bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1 \
                and bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1
        else:
            if (i, j) in [(0, 0), (0, 6), (6, 0), (6, 6)]:
                # safe in corner
                return False
            elif i == 0 or i == 6:
                # top or bottom: non corner
                return bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1
            elif j == 0 or j == 6:
                # left or right: non corner
                return bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1
            else:
                # any other spot on board
                if bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1:
                    return True
                elif bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1:
                    return True
                else:
                    return False
    else:
        msg = "piece_position[0] must be 0, 1, 2: {}".format(piece_position[0])
        raise Exception(msg)

def remove_captured_pieces(raw_board):
    """ take a raw (3, 7, 7) board and remove any
    captured pieces.
    
    Returns a raw_board """
    
    # get captured_pieces
    remaining_positions = [
        piece_position for piece_position in raw_board.indices
        if not is_piece_captured(raw_board, piece_position)
    ]
    
    return RawBoard(indices=remaining_positions,
                    skip_checks=raw_board.skip_checks)

In [123]:
class InvalidMoveError(Exception):
    def __init__(self):
        super().__init__("the piece cannot move there")
        
class NoPieceError(Exception):
    def __init__(self):
        super().__init__("that position doesn't have a piece to move")

def move(piece_position, new_position, board=None,
         game_state=None, game_history=None):
    """
    create a new board by moving the piece at
    piece_position to new_position and clearing
    off any captured pieces.
    
    piece_position and new_position should
    be (i, j, k) tuples.
    """
    
    if board is not None:
        assert game_state is None and game_history is None, \
            "only one of board, game_state, game_history can be provided"
    elif game_state is not None:
        assert game_history is None, \
            "only one of board, game_state, game_history can be provided"
        board = game_state.board
    else:
        assert game_history is not None, \
            "only one of board, game_state, game_history can be provided"
        assert len(game_history) != 0, "game_history cannot be empty"
        board = game_history[-1].board
    
    
    if board.raw_board[piece_position] == 0:
        raise NoPieceError
        
    if game_state is not None or game_history is not None:
        if game_state is not None:
            team = game_state.whose_turn
        else:
            team = game_history[-1].whose_turn
            
        if team == "attack":
            assert piece_position[0] == 0, \
                "attacker must move attacking pawns"
        else:
            assert piece_position[0] == 1 or piece_position[0] == 2, \
                "defender must move defending pawns or monarch"
    
    if not new_position in find_valid_moves(board, piece_position):
        raise InvalidMoveError
    
    # make the move
    new_positions = [p for p in board.positions if p != piece_position] \
        + [new_position]
    new_raw_board = RawBoard(indices=new_positions,
                             skip_checks=board.raw_board.skip_checks)
    
    # remove pieces
    clean_raw_board = remove_captured_pieces(new_raw_board)
    new_board = Board(clean_raw_board, board.fully_validate_board)
    
    if game_state is None and game_history is None:
        return new_board
    elif game_state is not None:
        if game_state.whose_turn == "attack":
            whose_turn = "defense"
        else:
            whose_turn = "attack"
        return GameState(new_board, whose_turn)
    else:
        game_state = game_history[-1]
        
        if game_state.whose_turn == "attack":
            whose_turn = "defense"
        else:
            whose_turn = "attack"
        new_game_state =  GameState(new_board, whose_turn)
        
        if len(game_history) == game_history.max_length:
            new_game_states = game_history.game_states[1:] + [new_game_state]
        else:
            new_game_states = game_history.game_states + [new_game_state]
        
        return GameHistory(new_game_states, game_history.max_length)
        

In [84]:
board = get_initial_board()
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [-1. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [85]:
board = move((0, 3, 0), (0, 2, 0), board)
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [86]:
board = move((1, 2, 3), (1, 2, 1), board)
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [87]:
board = move((0, 2, 0), (0, 1, 0), board)
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [88]:
board = move((1, 2, 1), (1, 1, 1), board)
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [89]:
board = move((0, 1, 3), (0, 1, 2), board)
print(board)

[[ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [90]:
board.check_victory('attack')

False

In [91]:
class GameState:
    """ this class enhances the board with some more gameplay aspects.
    also validates the validity of the board slightly. """
    
    def __init__(self, board, whose_turn):
        
        assert type(board) == Board
        self.__board = board
        
        assert not (self.attack_victory and self.defense_victory), \
            "only one of attack and defense can win"
        
        assert whose_turn in ["attack", "defense"], \
            "whose_turn must be 'attack' or 'defense'"
        self.__whose_turn = whose_turn
        
        
    def __eq__(self, other):
        """ equality based on board state """
        return self.board == other.board
    
    
    def __str__(self):
        return str(self.board.shadow_pretty)
        
    
    @property
    def board(self):
        return self.__board
    
    
    @property
    def possible_moves(self):
        if self.whose_turn == "attack":
            pos_index = [0]
        elif self.whose_turn == "defense":
            pos_index = [1, 2]
        else:
            raise InvalidTeamNameError
        
        return [
            (p, pp)
            for p in self.board.positions if p[0] in pos_index
            for pp in find_valid_moves(self.board, p)
        ]
    
    
    @property
    def attack_victory(self):
        return self.__check_victory("attack")
    
    
    @property
    def defense_victory(self):
        return self.__check_victory("defense")
    
    
    @property
    def has_winner(self):
        return self.attack_victory or self.defense_victory
        
    
    @property
    def whose_turn(self):
        return self.__whose_turn
    
    
    def __check_victory(self, team):
        return self.board.check_victory(team)

In [95]:
game_state = GameState(get_initial_board(), "attack")
print("initial state:")
print(game_state)
game_state = move((0, 3, 0), (0, 2, 0), game_state=game_state)
print("\nafter move:")
print(game_state)

initial state:
[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [-1. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]

after move:
[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [134]:
class GameHistory:
    """ hold n GameState objects and check if the game is over
    via a draw or victory """
    
    def __init__(self, game_states, max_length=5):
        
        assert max_length > 0, "max_length must be positive"
        self.__max_length = max_length
        
        assert type(game_states) == list, "game_states must bve list"
        assert len(game_states) <= self.max_length, \
            "can't have more gamme states than max_length"
        assert len(game_states) > 0, \
            "must have at lesat one initial game state"
        assert all(type(state) == GameState for state in game_states), \
            "all elements of game_states. must be GameState"
        if len(game_states) > 1:
            assert all(not state.has_winner for state in game_states[:-1]), \
                "only final game state can have a winner"
            
        self.__game_states = game_states
        
        
    def __getitem__(self, *args):
        return self.game_states.__getitem__(*args)
    
    
    def __len__(self):
        return len(self.game_states)
        
        
    @property
    def current_state(self):
        return self[-1]
        
        
    @property
    def game_states(self):
        return self.__game_states
    
    
    @property
    def attack_victory(self):
        return self.game_states[-1].attack_victory
    
    
    @property
    def defense_victory(self):
        return self.game_states[-1].defense_victory
    
    
    @property
    def has_winner(self):
        return self.game_states[-1].has_winner
    
    
    @property
    def is_draw(self):
        if len(self.game_states) < 2:
            return False
        else:
            return self.__has_draw(self.game_states)
        
        
    @property
    def max_length(self):
        return self.__max_length
            
    
    @classmethod
    def __has_draw(cls, game_states):
        """ recursively check for equal game states between the
        last element of the game_states list and the other elements.
        
        WARNING: recursive function """
        assert len(game_states) > 1, "must have at least two games"
        
        has_draw = any(gs == game_states[-1] for gs in game_states[:-1])
        if has_draw:
            return has_draw
        elif len(game_states) > 2:
            return cls.__has_draw(game_states[:-1])
        else:
            return False

In [135]:
gs0 = GameState(get_initial_board(), "attack")
gh = GameHistory([gs0])

moves = [
    ((0, 3, 0), (0, 2, 0)),
    ((1, 2, 3), (1, 2, 1)),
    ((0, 2, 0), (0, 1, 0)),
    ((1, 2, 1), (1, 1, 1)),
    ((0, 1, 3), (0, 1, 2)),
    ((1, 3, 4), (1, 2, 4)),
    ((0, 1, 0), (0, 5, 0)),
    ((1, 2, 4), (1, 3, 4)),
    ((0, 5, 0), (0, 1, 0))
]

for m in moves:
    try:
        gh = move(m[0], m[1], game_history=gh)
    except:
        print(m)
        raise
    if gh.has_winner or gh.is_draw:
        if gh.defense_victory:
            print("The defense wins!")
        elif gh.attack_victory:
            print("The attackers win!")
        else:
            print("draw!")
        print(gh.current_state)

draw!
[[ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  2.  1. -1. -1.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [139]:
gh = GameHistory([GameState(get_initial_board(), whose_turn="attack")])

moves = [
    ((0, 3, 0), (0, 1, 0)),
    ((1, 3, 2), (1, 2, 2)),
    ((0, 3, 1), (0, 2, 1)),
    ((2, 3, 3), (2, 3, 0)),
    ((0, 2, 1), (0, 5, 1)),
    ((2, 3, 0), (2, 6, 0))
]

for m in moves:
    try:
        gh = move(m[0], m[1], game_history=gh)
    except:
        print(m)
        raise
    if gh.has_winner or gh.is_draw:
        if gh.defense_victory:
            print("The defense wins!")
        elif gh.attack_victory:
            print("The attackers win!")
        else:
            print("draw!")
        print(gh.current_state)

The defense wins!
[[ 0.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.  0.]
 [ 2.  0.  0. -1.  0.  0.  0.]]


In [140]:
gh = GameHistory([GameState(get_initial_board(), whose_turn="attack")])

moves = [
    ((0, 3, 0), (0, 2, 0)),
    ((1, 3, 2), (1, 1, 2)),
    ((0, 3, 1), (0, 4, 1)),
    ((2, 3, 3), (2, 3, 0)),
    ((0, 4, 1), (0, 4, 0))
]

for m in moves:
    try:
        gh = move(m[0], m[1], game_history=gh)
    except:
        print(m)
        raise
    if gh.has_winner or gh.is_draw:
        if gh.defense_victory:
            print("The defense wins!")
        elif gh.attack_victory:
            print("The attackers win!")
        else:
            print("draw!")
        print(gh.current_state)

The attackers win!
[[ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1. -1.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


So far we've tried to stay as functional as possible. However, it really makes sense to 